# Tokenizers

Please can I bring you back to the wonderful Google Colab where we'll look at different Tokenizers:

https://colab.research.google.com/drive/1WD6Y2N7ctQi1X9wa6rpkg8UfyA4iSVuz?usp=sharing

# Exception - 
Instead of running this on collab. We will run this on Local machine

In [1]:
from huggingface_hub import login
from transformers import AutoTokenizer
import os 
from dotenv import load_dotenv 

In [2]:
# Log in to Hugging Face
load_dotenv(override=True)
hf_token = os.getenv('HF_TOKEN')
if hf_token and hf_token.startswith("hf_"):
  print("HF key looks good so far")
else:
  print("HF key is not set - please click the key in the left sidebar")
login(hf_token, add_to_git_credential=True)

HF key looks good so far


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
# Check Google Colab GPU - This is not needed on Macbook
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
  if gpu_info.find('Tesla T4') >= 0:
    print("Success - Connected to a T4")
  else:
    print("NOT CONNECTED TO A T4")

zsh:1: command not found: nvidia-smi
NOT CONNECTED TO A T4


Llama access was granted on Hugging face in less than few minutes 

# Accessing Llama 3.1 from Meta

In order to use the fantastic Llama 3.1, Meta does require you to sign their terms of service.

Visit their model instructions page in Hugging Face:
https://huggingface.co/meta-llama/Meta-Llama-3.1-8B

At the top of the page are instructions on how to agree to their terms. If possible, you should use the same email as your huggingface account.

In my experience approval comes in a couple of minutes. Once you've been approved for any 3.1 model, it applies to the whole 3.1 family of models. For whatever reason, occasionally Meta doesn't approve access. If that happens to you, please follow [this](https://colab.research.google.com/drive/1deJO03YZTXUwcq2vzxWbiBhrRuI29Vo8?usp=sharing) troubleshooting.

If the next cell gives you an error, then please check:  
1. Are you logged in to HuggingFace? Try running `login()` to check your key works
2. Did you set up your API key with full read and write permissions?
3. If you visit the Llama3.1 page with the link above, does it show that you have access to the model near the top?

I've also set up this troubleshooting colab to try to diagnose any HuggingFace connectivity issues:  
https://colab.research.google.com/drive/1deJO03YZTXUwcq2vzxWbiBhrRuI29Vo8?usp=sharing


# Get Token Details for Llama base model

Base Model in Hugging Face: "base model" is a pre-trained model that serves as a foundation for other, more specialized models, often a general model that has been fine-tuned for a specific task. It is a fundamental building block that can be customized through fine-tuning to perform new tasks like sentiment analysis, translation, or summarization. Examples include a base language model like bert-base-uncased or a base vision-language model. 

In [4]:
# Get the token details for llama pretrained "base model"
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3.1-8B', trust_remote_code=True)

In [5]:
# print the tokens generated for text below 
text = "I am excited to show Tokenizers in action to my LLM engineers"
tokens = tokenizer.encode(text)
tokens

[128000,
 40,
 1097,
 12304,
 311,
 1501,
 9857,
 12509,
 304,
 1957,
 311,
 856,
 445,
 11237,
 25175]

In [6]:
# get lengths 
character_count = len(text)
word_count = len(text.split(' '))
token_count = len(tokens)
print(f"There are {character_count} characters, {word_count} words and {token_count} tokens")

There are 61 characters, 12 words and 15 tokens


In [7]:
# now lets decode the tokens generated. Pay careful attention how llama inserts special tokens 
tokenizer.decode(tokens)

'<|begin_of_text|>I am excited to show Tokenizers in action to my LLM engineers'

In [8]:
# there is another way to see which token means which word in the sentence 
tokenizer.batch_decode(tokens)

['<|begin_of_text|>',
 'I',
 ' am',
 ' excited',
 ' to',
 ' show',
 ' Token',
 'izers',
 ' in',
 ' action',
 ' to',
 ' my',
 ' L',
 'LM',
 ' engineers']

In [9]:
# Get the vocabulary details of this tokenizer for special tokens
tokenizer.get_added_vocab()

{'<|begin_of_text|>': 128000,
 '<|end_of_text|>': 128001,
 '<|reserved_special_token_0|>': 128002,
 '<|reserved_special_token_1|>': 128003,
 '<|finetune_right_pad_id|>': 128004,
 '<|reserved_special_token_2|>': 128005,
 '<|start_header_id|>': 128006,
 '<|end_header_id|>': 128007,
 '<|eom_id|>': 128008,
 '<|eot_id|>': 128009,
 '<|python_tag|>': 128010,
 '<|reserved_special_token_3|>': 128011,
 '<|reserved_special_token_4|>': 128012,
 '<|reserved_special_token_5|>': 128013,
 '<|reserved_special_token_6|>': 128014,
 '<|reserved_special_token_7|>': 128015,
 '<|reserved_special_token_8|>': 128016,
 '<|reserved_special_token_9|>': 128017,
 '<|reserved_special_token_10|>': 128018,
 '<|reserved_special_token_11|>': 128019,
 '<|reserved_special_token_12|>': 128020,
 '<|reserved_special_token_13|>': 128021,
 '<|reserved_special_token_14|>': 128022,
 '<|reserved_special_token_15|>': 128023,
 '<|reserved_special_token_16|>': 128024,
 '<|reserved_special_token_17|>': 128025,
 '<|reserved_special_to

In [10]:
len(tokenizer.vocab)

128256

### This means there are 128000 words with token values and 256 special tokens  

# Instruct variant of the model 
IMany models have a variant that has been trained for use in Chats.
These are typically labelled with the word "Instruct" at the end.
They have been trained to expect prompts with a particular format that includes system, user and assistant prompts.

There is a utility method ```apply_chat_template``` that will convert from the messages list format we are familiar with, into the right input prompt for this model.

In [11]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3.1-8B-Instruct', trust_remote_code=True)

In [12]:
# construct the message format you want 
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Tell a light-hearted joke for a room of Data Scientists"}
  ]

# apply this as template for model to respond in this way
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>

Tell a light-hearted joke for a room of Data Scientists<|eot_id|><|start_header_id|>assistant<|end_header_id|>




```
LLM is just a Data Science model that takes a sequence of numbers and predicts the probability of the next number! You can't pass a bunch of Python objects into a statistical model!

And now you have the missing piece of the puzzle..
The messages in OpenAI format get converted:

1. ...into a sequence of words with special tags to separate the System, User, Assistant prompt
2. ...then the words are broken down into fragments - "tokens"
3. ...then the tokens are replaced with Token IDs - and this is the input sequence
The input to an LLM is a sequence of Token IDs. The output is the probability distribution of the next Token ID to follow this input.



# More Models

In [13]:
PHI4 = "microsoft/Phi-4-mini-instruct"
DEEPSEEK = "deepseek-ai/DeepSeek-V3.1"
QWEN_CODER = "Qwen/Qwen2.5-Coder-7B-Instruct"

# ATTENTION - refer material on cloud LLM Engineering - Week 3 Day 3.ipynb. Below is taking way too long to get the tokenizer.json file

In [14]:
# model was giving me errors - so making them single process to avoid deadlocks 
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [15]:
# create ph4 tokenizer 
phi4_tokenizer = AutoTokenizer.from_pretrained(PHI4)

# give a text 
text = "I am curiously excited to show Hugging Face Tokenizers in action to my LLM engineers"

# Print llama details 
print("Llama:")
tokens = tokenizer.encode(text)
print(tokens)
print(tokenizer.batch_decode(tokens))

# Print PHI4 details for same text
print("\nPhi 4:")
tokens = phi4_tokenizer.encode(text)
print(tokens)
print(phi4_tokenizer.batch_decode(tokens))


tokenizer.json:   0%|          | 0.00/15.5M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
deepseek_tokenizer = AutoTokenizer.from_pretrained(DEEPSEEK)

text = "I am curiously excited to show Hugging Face Tokenizers in action to my LLM engineers"
print(tokenizer.encode(text))
print()
print(phi4_tokenizer.encode(text))
print()
print(deepseek_tokenizer.encode(text))